In [ ]:
import copy
import numpy as np
import laspy as lp
import pyvista as pv
import pandas as pd

def view_cloud_pv(points):
    xyz = points[:, :3]  # Primeras 3 columnas: X, Y, Z
    rgb = points[:, 3:6]  # Columnas 4-6: R, G, B
    
    # Normalizar RGB de 16-bit (LAS) a 8-bit (0-255)
    if np.issubdtype(rgb.dtype, np.integer) and rgb.max() > 255:
        rgb = (rgb // 256).astype(np.uint8)  # Conversión a 8-bit
    
    # Crear objeto PolyData con coordenadas
    cloud = pv.PolyData(xyz)
    
    # Añadir colores RGB como array asociado a los puntos
    cloud["RGB"] = rgb.astype(np.uint8)  # Asegurar tipo uint8
    
    # Configurar plotter
    plotter = pv.Plotter()
    plotter.add_mesh(
        cloud,
        scalars="RGB",  # Usar el array RGB
        rgb=True,       # Indicar que son canales RGB
        point_size=3,   # Tamaño de los puntos
        opacity=1       # Opacidad total
    )
    
    plotter.show()


def generate_dem(points, rasante, grid_spacing=None):
    # Extraer y procesar coordenadas
    x, y, z = points[:, 0], points[:, 1], points[:, 2] - rasante
    #z = np.clip(z, 0, 4)

    # Calcular espaciado de la grilla
    if grid_spacing is None:
        dx = np.diff(x)
        dy = np.diff(y)
        point_spacing = np.mean(np.sqrt(dx**2 + dy**2))
        grid_spacing = point_spacing

    cols = int(np.ceil((x_max - x_min) / grid_spacing))
    rows = int(np.ceil((y_max - y_min) / grid_spacing))

    # Binning para promedio de elevaciones
    grid_sum = np.zeros((rows, cols))
    grid_count = np.zeros((rows, cols))
    
    xi = ((x - x_min) / grid_spacing).astype(int)
    yi = ((y_max - y) / grid_spacing).astype(int)
    
    np.add.at(grid_sum, (yi, xi), z)
    np.add.at(grid_count, (yi, xi), 1)
    
    dem = np.divide(grid_sum, grid_count, where=grid_count != 0)
    dem[grid_count == 0] = np.nan

    return dem

def get_points(path):
    las = lp.read(path)
    points = np.vstack((las.x, las.y, las.z)).transpose()
    rgb = np.vstack((las.red, las.green, las.blue)).transpose()
    data_points = np.hstack((
        points,
        rgb,
        np.expand_dims(las.intensity, -1),
        np.expand_dims(las.return_number, -1),
        np.expand_dims(las.number_of_returns, -1)
    ))
    
    single = data_points[data_points[:, -1] == 1][:, :6]  #XYZRGB
    return single 

def get_roi(points, p=0.05):
    # Obtener límites originales
    x_min = np.min(points[:, 0])
    x_max = np.max(points[:, 0])
    y_min = np.min(points[:, 1])
    y_max = np.max(points[:, 1])
    
    # Calcular 5% del largo de cada eje
    x_offset = p * (x_max - x_min)
    y_offset = p * (y_max - y_min)
    
    # Ajustar límites hacia adentro
    new_x_min = x_min + x_offset
    new_x_max = x_max - x_offset
    new_y_min = y_min + y_offset
    new_y_max = y_max - y_offset
    
    return new_x_min, new_x_max, new_y_min, new_y_max

In [15]:
df = pd.read_csv('data/vuelos/data_sqm.csv')

POZAS = {}

for index, row in df.iterrows():
    fecha = row['Fecha']
    poza = row['Poza'].replace('-','').lower()
    rasante = row['Rasante (m)']
    cota_salm = row['Cota Salm. (m)']
    cota_sal = row['Cota Sal (m)']
    sal = row['Sal (cm)']
    salmuera = row['Salmuera (cm)']

    if poza not in POZAS:
        POZAS[poza] = {}

    POZAS[poza][fecha] = {
        'Rasante (m)': rasante,
        'Cota Salmuera sqm (m)': cota_salm,
        'Cota Sal sqm (m)': cota_sal,
        'Sal sqm (cm)': sal,
        'Salmuera sqm (cm)': salmuera
    }

POZA = '5a'
DATE = '04_01_2025'
H = '100'
V = '10'
F = '50'
PATH_LAS = f'data/vuelos/{DATE}/{H}m_{V}ms_{50}khz/{POZA}_{H}m_{V}ms_{F}khz.las'
PDATA = POZAS.get(POZA).get(f'{DATE.replace("_","/")}')

print(f'Poza: {PATH_LAS}')
print('Data SQM')
PDATA

Poza: data/vuelos/04_01_2025/100m_10ms_50khz/5a_100m_10ms_50khz.las
Data SQM


{'Rasante (m)': 2300.29,
 'Cota Salmuera sqm (m)': 2300.457,
 'Cota Sal sqm (m)': 2300.29,
 'Sal sqm (cm)': 0.0,
 'Salmuera sqm (cm)': 15.87}

In [ ]:

points = get_points(PATH_LAS)
x_min, x_max, y_min, y_max = get_roi(points)
mask = (points[:, 0] >= x_min) & (points[:, 0] <= x_max) & \
       (points[:, 1] >= y_min) & (points[:, 1] <= y_max)
roi_points = points[mask]
print(roi_points.shape)
view_cloud_pv(points)
view_cloud_pv(roi_points)
dem_a = generate_dem(roi_points, RASANTE, , grid_spacing=0.17)


(4586665, 6)


Widget(value='<iframe src="http://localhost:42639/index.html?ui=P_0x762b65ad35e0_2&reconnect=auto" class="pyvi…

Widget(value='<iframe src="http://localhost:42639/index.html?ui=P_0x762b2d1cb010_3&reconnect=auto" class="pyvi…

In [ ]:
perimetro = {
    '3a':{
        'p1': (559353.270, 7394639.900),
        'p2': (559353.270, 7394912.350),
        'p3': (559591.970, 7394912.350),
        'p4': (559591.970, 7394639.900)
    },
    '5a':{
        'p1': (559871.270, 7394639.900),	
        'p2': (559871.270, 7394912.350),
        'p3': (560109.970, 7394912.350),
        'p4': (560109.970, 7394639.900)
    },
    '7a':{
        'p1': (560449.270, 7394639.900),
        'p2': (560449.270, 7394912.350),	
        'p3': (560747.970, 7394912.350),	
        'p4': (560747.970, 7394639.900)
    }
}

rasantes = {
    '3a': 2300.326,
    '5a': 2300.290,  
    '6a': 2300.260,   
    '7a': 2300.437,
}

salmueras = {
    '3a-3003': 2300.978,
    '5a-3103': 2300.455,
    '6a-3103': 2300.909,
    '7a-3103': 2301.189,
    
    '3a-1404': 2300.953,
    '5a-0104': 2300.457,
    '6a-0104': 2300.909,
    '7a-0104': 2301.194,
}

sales = {
    '3a-3003': 2300.757,
    '5a-3103': 2300.290,
    '6a-3103': 2300.804,
    '7a-3103': 2300.991,
    
    '3a-1404': 2300.790,
    '5a-0104': 2300.298,
    '6a-0104': 2300.930,
    '7a-0104': 2300.814,

}

asalmuera = {
    '3a-3003': 22.10,
    '5a-3103': 16.50,
    '6a-3103': 8.92,
    '7a-3103': 19.21,
    
    '3a-1404': 15.81,
    '5a-0104': 15.87,
    '6a-0104': 8.90,
    '7a-0104': 20.00,

}


POZA = '5a'
FECHAV1 = '3103'
FECHAV2 = '0104'
THRESHOLD = 0.02

RASANTE = rasantes.get(POZA, None)

SALMUERAV1 = salmueras.get(f'{POZA}-{FECHAV1}', None)
SALMUERAV2 = salmueras.get(f'{POZA}-{FECHAV2}', None)

SALV1 = sales.get(f'{POZA}-{FECHAV1}', None)
SALV2 = sales.get(f'{POZA}-{FECHAV2}', None)

ASALMUERAV1 = asalmuera.get(f'{POZA}-{FECHAV1}', None)
ASALMUERAV2 = asalmuera.get(f'{POZA}-{FECHAV2}', None)

print('salmuera')
print(SALMUERAV1, SALMUERAV2)
print('sales')
print(SALV1, SALV2)
print('altura sal')
print(ASALMUERAV1, ASALMUERAV2)

TRANS_INIT = np.asarray([[1.0, 0.0, 0.0, 0.0], 
                         [0.0, 1.0, 0.0, 0.0],
                         [0.0, 0.0, 1.0, 0.0], 
                         [0.0, 0.0, 0.0, 1.0]])

In [ ]:
source_path = 'data/03_31_2025/5a_100m_7ms_100khz_plena-luz_0_0_poza.las'
target_path = 'data/04_01_2025/5a_100m_10ms_100khz_plena-luz_0_0_poza.las'

In [ ]:
src_ground_points = get_ground_points(source_path, perimetro, POZA) 
src_ground_points = src_ground_points[:,:3]
pcd_src_ground_points = to_open3d(src_ground_points)

dst_ground_points = get_ground_points(target_path, perimetro, POZA) 
dst_ground_points = dst_ground_points[:,:3]
pcd_dst_ground_points = to_open3d(dst_ground_points)

#VOXEL_SIZE = 1.0  
#THRESHOLD = VOXEL_SIZE * 1.5
#TRANS_INIT = fast_global_registration(pcd_src_ground_points, pcd_dst_ground_points, VOXEL_SIZE) 

pcd_src_ground_points.estimate_normals()
pcd_dst_ground_points.estimate_normals()

print("Apply point-to-plane ICP")
reg_p2l = o3d.pipelines.registration.registration_icp(
            pcd_src_ground_points, 
            pcd_dst_ground_points, 
            THRESHOLD, 
            TRANS_INIT,
            o3d.pipelines.registration.TransformationEstimationPointToPlane()
        )


transformation = copy.deepcopy(reg_p2l.transformation)

print("Transformation is:")
print(transformation)

las = lp.read(source_path)
src_las_data_points_transformed = np.dot(transformation[:3, :3], las.xyz.T).T + transformation[:3, 3]

las.x = src_las_data_points_transformed[:, 0]
las.y = src_las_data_points_transformed[:, 1]
las.z = src_las_data_points_transformed[:, 2]

source_las_transformed = source_path.replace('.las', '_transformed.las')
print(f"Saving transformed LAS to {source_las_transformed}")
las.write(source_las_transformed)

In [ ]:
points_a = get_point(source_las_transformed, perimetro, POZA)[:,:3]
points_b = get_point(target_path, perimetro, POZA)[:,:3]

xmin, xmax = np.min([np.min(points_a[:,0]), np.min(points_b[:,0])]), np.max([np.max(points_a[:,0]), np.max(points_b[:,0])])
ymin, ymax = np.min([np.min(points_a[:,1]), np.min(points_b[:,1])]), np.max([np.max(points_a[:,1]), np.max(points_b[:,1])])


dem_b = generate_dem(points_b, RASANTE, xmin, xmax, ymin, ymax, grid_spacing=0.17)

In [ ]:
# import matplotlib.pyplot as plt 
# points_a = get_point(source_las_transformed, perimetro, POZA)[:,:3]
# points_b = get_point(target_path, perimetro, POZA)
# mask_intensity = points_b[:,3] > 48500 
# points_b = points_b[mask_intensity]
# plt.hist(points_b[:,3], bins=30)
# plt.show()
# plt.hist(points_b[:,2], bins=30)
# plt.show()
# points_b = points_b[:,:3]

# xmin, xmax = np.min([np.min(points_a[:,0]), np.min(points_b[:,0])]), np.max([np.max(points_a[:,0]), np.max(points_b[:,0])])
# ymin, ymax = np.min([np.min(points_a[:,1]), np.min(points_b[:,1])]), np.max([np.max(points_a[:,1]), np.max(points_b[:,1])])

# dem_a = generate_dem(points_a, RASANTE, xmin, xmax, ymin, ymax, grid_spacing=0.17)
# dem_b = generate_dem(points_b, RASANTE, xmin, xmax, ymin, ymax, grid_spacing=0.17)

In [ ]:
ASALV1 = np.mean(points_a[:,2] - RASANTE)*100
ASALV2 = np.mean(points_b[:,2] - RASANTE)*100

print(f'vuelo {FECHAV1} sal promedio: {ASALV1}')
print(f'vuelo {FECHAV2} sal promedio: {ASALV2}')

xx =  dem_b - dem_a 
xx = xx[~np.isnan(xx)]
print(f'diferencia promedio: {np.mean(xx)*100}')

In [ ]:

import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
plt.imshow(dem_a, cmap='jet')
plt.colorbar()
plt.title(f'Altura sal vuelo {FECHAV1} {POZA}')  
plt.axis('off')
plt.show()

plt.figure(figsize=(20,10))
plt.imshow(dem_b, cmap='jet')
plt.colorbar()
plt.title(f'Altura sal vuelo {FECHAV2} {POZA}')  
plt.axis('off')
plt.show()

plt.figure(figsize=(20,10))
plt.imshow(dem_b - dem_a, cmap='jet')
plt.colorbar()
plt.axis('off')
plt.title(f'Diferencia entre vuelos {FECHAV1} - {FECHAV2} {POZA}')  
plt.show()


In [ ]:
def bresenham_line(start=(10, 10), end=(10, 60)):
    # Unpack start and end points
    x1, y1 = np.array(start, dtype=np.int32).copy()
    x2, y2 = np.array(end, dtype=np.int32).copy()


    # Calculate differences
    dx = x2 - x1
    dy = y2 - y1

    # Determine if the line is steep (more vertical than horizontal)
    is_steep = abs(dy) > abs(dx)

    # Swap coordinates if the line is steep (for easier handling)
    if is_steep:
        x1, y1 = y1, x1
        x2, y2 = y2, x2

    # Ensure the line is always drawn left-to-right
    if x1 > x2:
        x1, x2 = x2, x1
        y1, y2 = y2, y1

    # Recalculate differences after the possible swap
    dx = x2 - x1
    dy = y2 - y1

    # Error term initialized to half of dx
    error = dx // 2
    ystep = 1 if y1 < y2 else -1  # Determines whether to increment or decrement y

    # List to store the generated points
    points = []
    y = y1

    # Main loop for Bresenham's algorithm
    for x in range(x1, x2 + 1):
        coord = (y, x) if is_steep else (x, y)  # Swap x and y if the line is steep
        points.append(coord)

        # Update error term
        error -= abs(dy)

        # If error is negative, adjust y and reset the error term
        if error < 0:
            y += ystep
            error += dx

    return points

h, w = dem_a.shape
line = bresenham_line(start=(0, 0), end=(h-1,w-1))

In [ ]:
from numpy.polynomial.polynomial import Polynomial

z_d1 = np.array([(idx, dem_a[x][y]) for idx, (x, y) in enumerate(line) if not np.isnan(dem_a[x][y])])
z_d2 = np.array([(idx, dem_b[x][y]) for idx, (x, y) in enumerate(line) if not np.isnan(dem_b[x][y])])
print(z_d1.shape, z_d2.shape)

degree = 30
poly_d1 = Polynomial.fit(z_d1[:,0], z_d1[:,1], degree)
poly_d2 = Polynomial.fit(z_d2[:,0], z_d2[:,1], degree)

x_new_d1 = z_d1[:,0]
x_new_d2 = z_d2[:,0]
z_new_d1 = poly_d1(x_new_d1)
z_new_d2 = poly_d2(x_new_d2)

mean_diagonal1 = np.mean(z_d1[:,1])
mean_diagonal2 = np.mean(z_d2[:,1])

plt.figure(figsize=(20, 6))
plt.scatter(z_d1[:,0], z_d1[:,1],color='blue' ,label=f'Diagonal Principal {FECHAV1} ')
plt.scatter(z_d2[:,0], z_d2[:,1],color='red', label=f'Diagonal Principal {FECHAV2}')

plt.axhline(0, color='black', linestyle='--', linewidth=1)
plt.axhline(mean_diagonal1, color='blue', linestyle='--', linewidth=1, label=f'Media Diagonal {FECHAV1}: {mean_diagonal1:.4f}')
plt.axhline(mean_diagonal2, color='red', linestyle='--', linewidth=1, label=f'Media Diagonal {FECHAV2}: {mean_diagonal2:.4f}')
plt.axhline(SALMUERAV1-RASANTE, color='black', linestyle='--', linewidth=1, label=f'salmuera {FECHAV1}')
plt.axhline(SALMUERAV2-RASANTE, color='green', linestyle='--', linewidth=1, label=f'salmuera {FECHAV2}')
plt.plot(x_new_d1, z_new_d1, label=f'Diagonal Principal {FECHAV1}')
plt.plot(x_new_d2, z_new_d2, label=f'Diagonal Principal {FECHAV2}')
plt.title(f'Poza {POZA} {FECHAV1} vs {FECHAV2}')
plt.ylabel('Altura Sal (m)')
plt.xticks([])
plt.legend()
plt.show()

In [ ]:
from PIL import Image
import json

OUTPUTS_PATH = 'data/results'

pil_image_z = Image.fromarray(dem_a)
pil_image_z.save(f'{OUTPUTS_PATH}/{POZA}_{FECHAV1}.tiff')

pil_image_z = Image.fromarray(dem_b)
pil_image_z.save(f'{OUTPUTS_PATH}/{POZA}_{FECHAV2}.tiff')

for dem, date, csalmuera, csal, asal, asalmuera in zip([dem_a, dem_b], [FECHAV1, FECHAV2], [SALMUERAV1, SALMUERAV2],
                                                       [SALV1, SALV2], [ASALV1, ASALV2], [ASALMUERAV1, ASALMUERAV2]):
    h, w = dem.shape
    d1 = bresenham_line(start=(0, 0), end=(h-1,w-1))
    d2 = bresenham_line(start=(h-1, 0), end=(0,w-1))
    dem_d1 = np.array([(idx, dem[x][y]) for idx, (x, y) in enumerate(d1) if not np.isnan(dem[x][y])])
    dem_d2 = np.array([(idx, dem[x][y]) for idx, (x, y) in enumerate(d2) if not np.isnan(dem[x][y])])

    result = {
        'altura sal': asal,
        'altura salmuera': asalmuera,
        'cota salmuera': csalmuera,
        'cota sal':csal,
        'rasante': RASANTE,
        'diagonal1':{
            'unit':     '[m]',
            'mean':     float(np.mean(dem_d1[:,1])),
            'x_axis':   dem_d1[:,0].tolist(),
            'y_axis':   dem_d1[:,1].tolist()

        },
        'diagonal2':{
            'unit':     '[m]',
            'mean':     float(np.mean(dem_d2[:,1])),
            'x_axis':   dem_d2[:,0].tolist(),
            'y_axis':   dem_d2[:,1].tolist()
        }
    }

    with open(f'{OUTPUTS_PATH}/{POZA}_{date}.json', 'w') as f:
        json.dump(result, f, indent=4)
